In [16]:
# symmetric encryption example

from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP, AES
from Crypto.Random import get_random_bytes
from Crypto.Hash import SHA256, HMAC
import base64
import os

# Generate RSA Keys for a user
def generate_rsa_keys():
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    return private_key, public_key

# Encrypt file using the user's public key
def encrypt_file(file_data, public_key):
    rsa_key = RSA.import_key(public_key)
    cipher_rsa = PKCS1_OAEP.new(rsa_key)
    encrypted_file = cipher_rsa.encrypt(file_data)
    return base64.b64encode(encrypted_file).decode()

# Encrypt metadata using the shared symmetric key
def encrypt_metadata(metadata, symmetric_key):
    nonce = get_random_bytes(12)
    cipher_aes = AES.new(symmetric_key, AES.MODE_GCM, nonce=nonce)
    encrypted_metadata, tag = cipher_aes.encrypt_and_digest(metadata.encode())
    return base64.b64encode(nonce + encrypted_metadata).decode()

# Generate search tokens for prefixes using the shared symmetric key
def generate_search_tokens(keyword, symmetric_key):
    tokens = []
    for i in range(1, len(keyword) + 1):
        prefix = keyword[:i]
        h = HMAC.new(symmetric_key, digestmod=SHA256)
        h.update(prefix.encode())
        tokens.append(h.hexdigest())
    return tokens

# Perform search using the search tokens
def search(encrypted_metadata_list, search_tokens, symmetric_key):
    for encrypted_metadata in encrypted_metadata_list:
        encrypted_metadata_bytes = base64.b64decode(encrypted_metadata)
        nonce = encrypted_metadata_bytes[:12]
        ciphertext = encrypted_metadata_bytes[12:]
        try:
            cipher_aes = AES.new(symmetric_key, AES.MODE_GCM, nonce=nonce)
            decrypted_metadata = cipher_aes.decrypt(ciphertext).decode()
            print(f"Decrypted metadata: {decrypted_metadata}")  # Debugging
            for token in search_tokens:
                h = HMAC.new(symmetric_key, digestmod=SHA256)
                h.update(decrypted_metadata.encode())
                metadata_token = h.hexdigest()
                if metadata_token == token:
                    return decrypted_metadata
        except Exception as e:
            print(f"Decryption failed: {e}")  # Debugging
            continue
    return None

# Decrypt file using the user's private key
def decrypt_file(encrypted_file, private_key):
    rsa_key = RSA.import_key(private_key)
    cipher_rsa = PKCS1_OAEP.new(rsa_key)
    decrypted_file = cipher_rsa.decrypt(base64.b64decode(encrypted_file))
    return decrypted_file

# Main function demonstrating the process
def main():
    # Generate keys for two users
    private_key_user1, public_key_user1 = generate_rsa_keys()
    private_key_user2, public_key_user2 = generate_rsa_keys()

    # Shared symmetric key for searchable encryption (must be securely shared)
    shared_symmetric_key = get_random_bytes(32)
    
    encrypted_metadata_list = []

    # Example file and metadata
    file_data = b"Example file content"
    metadata = "file:example"

    # User 1 encrypts the file and metadata
    encrypted_file = encrypt_file(file_data, public_key_user1)
    encrypted_metadata = encrypt_metadata(metadata, shared_symmetric_key)
    
    encrypted_metadata_list.append(encrypted_metadata)
    
     # Example file and metadata
    file_data = b"test 2 document data"
    metadata = "file:test2"

    # User 1 encrypts the file and metadata
    encrypted_file = encrypt_file(file_data, public_key_user1)
    encrypted_metadata = encrypt_metadata(metadata, shared_symmetric_key)
    
    encrypted_metadata_list.append(encrypted_metadata)
    


    # User 2 generates search tokens for the prefix "exa"
    search_keyword = "file:test2"
    search_tokens = generate_search_tokens(search_keyword, shared_symmetric_key)
    print(f"Search tokens: {search_tokens}")  # Debugging
    
    # User 2 searches for the metadata
    found_metadata = search(encrypted_metadata_list, search_tokens, shared_symmetric_key)
    if found_metadata:
        print(f"Metadata found: {found_metadata}")

        # User 2 requests the encrypted file and decrypts it with User 1's private key (assuming access is granted)
        decrypted_file = decrypt_file(encrypted_file, private_key_user1)
        print(f"Decrypted file content: {decrypted_file.decode()}")
    else:
        print("Metadata not found")

if __name__ == "__main__":
    main()


Search tokens: ['26c7f26719b50a7dc0868b71bf13feaec6d2c72beefb144533304b5028786ec1', '01edc288941e683a182a1a4a9fca27f3a7b21504a6c8a54cc5075f55789effaf', '0a457a31bd997a878b36ce3851d076c403749ebcc9930bf00d40087e78275bd6', 'fed3a48bd213476e5fcffae2e56718c606bf1536a28734cc43b45ee10386229b', 'ae865afee37214435b41a2df8610521b01c863bb27265d763300acebaa4c24a0', '6d101978caf36debce3f74c3df1c33b1fe4bfa356ed877c4f0a7abe5d2ebd8ce', '9315866456f218a94ae16ef4ad56746238f3d61bb8d671ae4803d1247fc155c6', 'db3d3696f618f6e32ab150ebeade41be389ab845b6ea7573f0c15001c1ba6e57', 'c720d490a256576929b7668ba3b160a11c41398c9576123830e8318d2936ab58', '54c491d14fec4661d8e51e4ac598e585e4a7fe3cb8d5fdd03eb7a831ab6f5828']
Decrypted metadata: file:example
Decrypted metadata: file:test2
Metadata found: file:test2
Decrypted file content: test 2 document data


In [22]:
# deterministic encryption example

from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Hash import SHA256
import base64
import os

# Generate RSA Keys for a user
def generate_rsa_keys():
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    return private_key, public_key

# Encrypt file using the user's public key
def encrypt_file(file_data, public_key):
    rsa_key = RSA.import_key(public_key)
    cipher_rsa = PKCS1_OAEP.new(rsa_key)
    encrypted_file = cipher_rsa.encrypt(file_data)
    return base64.b64encode(encrypted_file).decode()

# Deterministically encrypt metadata using the shared symmetric key
def encrypt_metadata_deterministic(metadata, symmetric_key):
    cipher = AES.new(symmetric_key, AES.MODE_ECB)
    hash_metadata = SHA256.new(metadata.encode()).digest()
    encrypted_metadata = cipher.encrypt(hash_metadata)
    return base64.b64encode(encrypted_metadata).decode()

# Generate search token for the keyword using the shared symmetric key
def generate_search_token(keyword, symmetric_key):
    cipher = AES.new(symmetric_key, AES.MODE_ECB)
    hash_keyword = SHA256.new(keyword.encode()).digest()
    search_token = cipher.encrypt(hash_keyword)
    return base64.b64encode(search_token).decode()

# Perform search using the search token
def search(encrypted_metadata_list, search_token):
    search_token_bytes = base64.b64decode(search_token)
    for encrypted_metadata in encrypted_metadata_list:
        encrypted_metadata_bytes = base64.b64decode(encrypted_metadata)
        if encrypted_metadata_bytes == search_token_bytes:
            return encrypted_metadata
    return None

# Decrypt file using the user's private key
def decrypt_file(encrypted_file, private_key):
    rsa_key = RSA.import_key(private_key)
    cipher_rsa = PKCS1_OAEP.new(rsa_key)
    decrypted_file = cipher_rsa.decrypt(base64.b64decode(encrypted_file))
    return decrypted_file

# Main function demonstrating the process
def main():
    # Generate keys for two users
    private_key_user1, public_key_user1 = generate_rsa_keys()
    private_key_user2, public_key_user2 = generate_rsa_keys()

    # Shared symmetric key for searchable encryption (must be securely shared)
    shared_symmetric_key = SHA256.new(b"shared_key").digest()

    encrypted_metadata_list = [];
    # Example file and metadata
    file_data = b"Example file content"
    metadata = "file:example"

    # User 1 encrypts the file and metadata
    encrypted_file = encrypt_file(file_data, public_key_user1)
    encrypted_metadata = encrypt_metadata_deterministic(metadata, shared_symmetric_key)

    # Simulate a list of encrypted metadata
    encrypted_metadata_list.append(encrypted_metadata)
    
    file_data = b"test file content"
    metadata = "file:test23"

    # User 1 encrypts the file and metadata
    encrypted_file = encrypt_file(file_data, public_key_user1)
    encrypted_metadata = encrypt_metadata_deterministic(metadata, shared_symmetric_key)
    
    encrypted_metadata_list.append(encrypted_metadata)


    # User 2 generates a search token for the keyword "example"
    search_keyword = "file:test23"
    search_token = generate_search_token(search_keyword, shared_symmetric_key)
    print(f"Search token: {search_token}")  # Debugging
    
    # User 2 searches for the metadata
    found_metadata = search(encrypted_metadata_list, search_token)
    if found_metadata:
        print(f"Metadata found: {found_metadata}")

        # User 2 requests the encrypted file and decrypts it with User 1's private key (assuming access is granted)
        decrypted_file = decrypt_file(encrypted_file, private_key_user1)
        print(f"Decrypted file content: {decrypted_file.decode()}")
    else:
        print("Metadata not found")

if __name__ == "__main__":
    main()


Search token: NwxqtI5t2ZFHvh5JzmUf+GMcslePCwzXn2C6KmCDC3k=
Metadata found: NwxqtI5t2ZFHvh5JzmUf+GMcslePCwzXn2C6KmCDC3k=
Decrypted file content: test file content


In [55]:
import hashlib
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Protocol.KDF import PBKDF2

# Generate symmetric key from password
def generate_symmetric_key(password, salt):
    return PBKDF2(password, salt, dkLen=32, count=1000)

# Encrypt data with AES
def encrypt_data(data, key):
    cipher = AES.new(key, AES.MODE_GCM)
    nonce = cipher.nonce
    ciphertext, tag = cipher.encrypt_and_digest(data.encode())
    return nonce + ciphertext

# Decrypt data with AES
def decrypt_data(encrypted_data, key):
    nonce = encrypted_data[:16]
    ciphertext = encrypted_data[16:]
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    return cipher.decrypt(ciphertext).decode()

# Hash a searchable field
def hash_field(field):
    return hashlib.sha256(field.encode()).hexdigest()

# Sample data
data_records = [
    {"user_id": "jsmith45", "name": "John Smith", "dob": "12/3/1945", "pan": "4567-8901-2345-6789", "ssn": "001-23-4567"},
    {"user_id": "mpower", "name": "Max Power", "dob": "4/15/1997", "pan": "4285-9487-2345-6754", "ssn": "010-77-1143"},
    {"user_id": "it2235", "name": "Jill Turner", "dob": "1/1/1930", "pan": "4673-2126-5623-7596", "ssn": "489-21-3532"}
]

# Encrypt data records and create indices
password = "strongpassword"
salt = get_random_bytes(16)
key = generate_symmetric_key(password, salt)

encrypted_data_records = []
indices = []

for record in data_records:
    encrypted_record = {k: encrypt_data(v, key) for k, v in record.items()}
    encrypted_data_records.append(encrypted_record)
    indices.append({
        "name": hash_field(record["name"]),
        "pan_last4": record["pan"][-4:],
        "ssn_last4": hash_field(record["ssn"][-4:])
    })

print(indices)
# Searching the database
def search_database(name_hash=None, pan_last4=None, ssn_last4_hash=None):
    results = []
    for record, index in zip(encrypted_data_records, indices):
        print(record, index)
        if (name_hash and index["name"] == name_hash) or \
           (pan_last4 and index["pan_last4"] == pan_last4) or \
           (ssn_last4_hash and index["ssn_last4"] == ssn_last4_hash):
            results.append(record)
    return results

# Example search
search_name = hash_field("John")
print(search_name)
found_records = search_database(name_hash=search_name)
for record in found_records:
    decrypted_record = {k: decrypt_data(v, key) for k, v in record.items()}
    print(decrypted_record)


[{'name': 'ef61a579c907bbed674c0dbcbcf7f7af8f851538eef7b8e58c5bee0b8cfdac4a', 'pan_last4': '6789', 'ssn_last4': 'db2e7f1bd5ab9968ae76199b7cc74795ca7404d5a08d78567715ce532f9d2669'}, {'name': 'd2d9e129ef7cabd8afd68021feda19fee87308bd8e23987548d7559ac7635a53', 'pan_last4': '6754', 'ssn_last4': 'a183251a8414fd819da06488541d523365d51a7d0d942e272c2dce108f6dcdae'}, {'name': '49eefb9bc1f8e6d0533925565afd9ab17d3c6c03f30d57baf42123bc7ed112c7', 'pan_last4': '7596', 'ssn_last4': '529faed5f67da7f6bbd6d69f20dfea73ec00d1dbe324dcc2ee4e9f197dcf0a61'}]
a8cfcd74832004951b4408cdb0a5dbcd8c7e52d43f7fe244bf720582e05241da
{'user_id': b'\xc4\xf0\xb9T\xbe!3&\xae\xa2,fe\xaf\x8c\xed\xe4+\xa3\xe3\xb2o\x95C', 'name': b'fT`\x8d\xbb\xd9\xaaH\x7f\x91?FnyJ\xac\x08\xf5\xa6\x93dm\x1b\xffp_', 'dob': b'-\x18\xe5\xda\x04\xf2\r\x0c\x8d[\xaeB\x82\xecB\x84\xb25#\x1b\x8f\xce?a8', 'pan': b'?L`c+\x8cN8\xbdS\x82\xd2[\xe9$\xfe\x92\xb7\xa8\xa5/\x82\x9ay.n\x97\x9d\xdd\xc1^\xb7\xb79\x95', 'ssn': b'87\x84\xb5f;\x8d\x9f\x92\x14\x8f\xc4\

In [70]:
import hashlib
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Protocol.KDF import PBKDF2

# Generate a symmetric key from a password
def generate_symmetric_key(password, salt):
    return PBKDF2(password, salt, dkLen=32, count=1000)

# Encrypt data with AES
def encrypt_data(data, key):
    cipher = AES.new(key, AES.MODE_GCM)
    nonce = cipher.nonce
    ciphertext, tag = cipher.encrypt_and_digest(data.encode())
    return nonce + ciphertext

# Decrypt data with AES
def decrypt_data(encrypted_data, key):
    nonce = encrypted_data[:16]
    ciphertext = encrypted_data[16:]
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    return cipher.decrypt(ciphertext).decode()

# Hash a field
def hash_field(field):
    return hashlib.sha256(field.encode()).hexdigest()

# Generate all prefixes of a string
def generate_prefixes(field):
    return [field[:i] for i in range(1, len(field) + 1)]

# Sample data
data_records = [
    {"user_id": "jsmith45", "name": "John Smith", "dob": "12/3/1945", "pan": "4567-8901-2345-6789", "ssn": "001-23-4567"},
    {"user_id": "mpower", "name": "Max Power", "dob": "4/15/1997", "pan": "4285-9487-2345-6754", "ssn": "010-77-1143"},
    {"user_id": "it2235", "name": "Joy Tthin", "dob": "1/1/1930", "pan": "4673-2126-5623-7596", "ssn": "489-21-3532"}
]

# Encrypt data records and create indices
password = "strongpassword"
salt = get_random_bytes(16)
key = generate_symmetric_key(password, salt)

encrypted_data_records = []
indices = []

for record in data_records:
    encrypted_record = {k: encrypt_data(v, key) for k, v in record.items()}
    encrypted_data_records.append(encrypted_record)
    record_indices = []
    for field, value in record.items():
        record_prefixes = generate_prefixes(value)
        record_indices.extend([hash_field(prefix) for prefix in record_prefixes])
    indices.append(record_indices)

# Searching the database
def search_database(query):
    search_token = hash_field(query)
    results = []
    for record, index in zip(encrypted_data_records, indices):
        if search_token in index:
            results.append(record)
    return results

# Example search for "Joh" to find "John Smith"
search_query = "th"
found_records = search_database(search_query)
for record in found_records:
    decrypted_record = {k: decrypt_data(v, key) for k, v in record.items()}
    print(decrypted_record)



In [138]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Protocol.KDF import PBKDF2
import hashlib
import base64

# Helper functions
def pad(data):
    pad_len = 16 - len(data) % 16
    return data + bytes([pad_len] * pad_len)

def unpad(data):
    pad_len = data[-1]
    return data[:-pad_len]

def derive_key(master_key, user_id):
    salt = hashlib.sha256(user_id.encode()).digest()
    return PBKDF2(master_key, salt, dkLen=32, count=1000)

def encrypt_data(sym_key, data):
    cipher = AES.new(sym_key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(data.encode()))
    return base64.b64encode(cipher.iv + ct_bytes).decode('utf-8')

def decrypt_data(sym_key, enc_data):
    enc_data = base64.b64decode(enc_data)
    iv = enc_data[:16]
    ct = enc_data[16:]
    cipher = AES.new(sym_key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(ct)).decode('utf-8')

def encrypt_keyword(key, keyword):
    cipher = AES.new(key, AES.MODE_ECB)
    padded_keyword = pad(keyword.encode())
    encrypted_keyword = cipher.encrypt(padded_keyword)
    return base64.b64encode(encrypted_keyword).decode('utf-8')

def encrypt_symmetric_key(user_key, sym_key):
    cipher = AES.new(user_key, AES.MODE_ECB)
    return base64.b64encode(cipher.encrypt(pad(sym_key))).decode('utf-8')

def decrypt_symmetric_key(user_key, enc_sym_key):
    cipher = AES.new(user_key, AES.MODE_ECB)
    sym_key_padded = cipher.decrypt(base64.b64decode(enc_sym_key))
    return unpad(sym_key_padded)

# Normalize text for consistent encryption
def normalize_text(text):
    return text.lower()

# Setup
master_key = get_random_bytes(32)

# User 1 and User 2 Keys
user1_id = 'user1'
user2_id = 'user2'
user1_key = derive_key(master_key, user1_id)
user2_key = derive_key(master_key, user2_id)
keyword_key = derive_key(master_key, 'keyword')

# User 1 encrypts data
texts = ["This is a sample text about Python",
         "Another text including Python and encryption",
         "A final sample text that talks about security"]

# Encrypt texts with a symmetric key and then encrypt the symmetric key
encrypted_texts = []
encrypted_sym_keys = []
for text in texts:
    sym_key = get_random_bytes(32)
    enc_text = encrypt_data(sym_key, text)
    enc_sym_key_user1 = encrypt_symmetric_key(user1_key, sym_key)
    enc_sym_key_user2 = encrypt_symmetric_key(user2_key, sym_key)
    encrypted_texts.append((enc_text, enc_sym_key_user1, enc_sym_key_user2))

# Create encrypted index for each text
encrypted_indices = []
for text in texts:
    encrypted_index = set()
    normalized_text = normalize_text(text)
    for word in normalized_text.split():
        encrypted_index.add(encrypt_keyword(keyword_key, word))
    encrypted_indices.append(encrypted_index)

# Function to search encrypted data
def search_encrypted_data(keyword_key, user_key, keyword, encrypted_texts, encrypted_indices):
    matches = []
    normalized_keyword = normalize_text(keyword)
    keyword_enc = encrypt_keyword(keyword_key, normalized_keyword)
    print(f'Searching for: {keyword}')
    print(f'Encrypted keyword: {keyword_enc}')
    for i, enc_index in enumerate(encrypted_indices):
        print(f'Checking index {i}: {enc_index}')
        if keyword_enc in enc_index:
            enc_text, enc_sym_key_user1, enc_sym_key_user2 = encrypted_texts[i]
            matches.append((enc_text, enc_sym_key_user2))
            print(f'Match found in text index: {i}')
    return matches

# User 2 searches for a keyword in encrypted data
search_keyword = "sample"
matching_texts = search_encrypted_data(keyword_key, user2_key, search_keyword, encrypted_texts, encrypted_indices)
decrypted_texts = []
for enc_text, enc_sym_key_user2 in matching_texts:
    sym_key = decrypt_symmetric_key(user2_key, enc_sym_key_user2)
    decrypted_texts.append(decrypt_data(sym_key, enc_text))

# Output
print("Encrypted Texts:")
print(encrypted_texts)
print("\nMatching Encrypted Texts:")
print(matching_texts)
print("\nDecrypted Texts:")
print(decrypted_texts)


Searching for: sample
Encrypted keyword: z0yDcHAtzMPzRFgKaywGJg==
Checking index 0: {'QSJnXtj0n0J60DvnZaOESQ==', 'z0yDcHAtzMPzRFgKaywGJg==', '6Ig97fX7Ps+UACD6f3GPew==', 'K//bO2AN3wXuLKA4k46RVQ==', 'oNAriImOMDZ9cFKUyYquMA==', 'ph4bUW52QQTLoDE3R5xsQQ==', 'M4+do0eQpSSRo7bMpMFowg=='}
Match found in text index: 0
Checking index 1: {'Pmqi27YadC58ShE++NZw+w==', 'UTDfWc3UBhqaGR3cYtH6rw==', 'K//bO2AN3wXuLKA4k46RVQ==', '6Ig97fX7Ps+UACD6f3GPew==', 'raM6YxRnFzRFjs10ZK1XfA==', 'XH8bfV2uShnDThQTJKbkcg=='}
Checking index 2: {'qMPtNvGk3N0PCX8zTgx2Og==', 'z0yDcHAtzMPzRFgKaywGJg==', '6Ig97fX7Ps+UACD6f3GPew==', 'ZGsZifDUmbUPKVwrMidNvw==', '1OM9Zd9jMDGSA1Dowj+WZw==', 'oNAriImOMDZ9cFKUyYquMA==', 'IziFGl2pR2OFJsY8zp1ifw==', 'M4+do0eQpSSRo7bMpMFowg=='}
Match found in text index: 2
Encrypted Texts:
[('9OHksy/8QufBZ7yegHT1hKF+V25FIsy0XTLDvNiJiHVu74GrGMZyooB85XGGLU3Jp+KAu1wIB0z+eY816BoKBw==', 'wIpm+GKzCgNmeSaco/lcASOTxkHC8+EyaYYZThrwBIKkiTpuvAVZuDxCP/mAX6+I', 'beldNWPSO+IH9TucAxzdco5rdB4zPd5MAzZ5Lu3R9E0dvwtPbWG

In [142]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Protocol.KDF import PBKDF2
import hashlib
import base64
from datetime import datetime

# Helper functions
def pad(data):
    pad_len = 16 - len(data) % 16
    return data + bytes([pad_len] * pad_len)

def unpad(data):
    pad_len = data[-1]
    return data[:-pad_len]

def derive_key(master_key, user_id):
    salt = hashlib.sha256(user_id.encode()).digest()
    return PBKDF2(master_key, salt, dkLen=32, count=1000)

def encrypt_data(sym_key, data):
    cipher = AES.new(sym_key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(data.encode()))
    return base64.b64encode(cipher.iv + ct_bytes).decode('utf-8')

def decrypt_data(sym_key, enc_data):
    enc_data = base64.b64decode(enc_data)
    iv = enc_data[:16]
    ct = enc_data[16:]
    cipher = AES.new(sym_key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(ct)).decode('utf-8')

def encrypt_keyword(key, keyword):
    cipher = AES.new(key, AES.MODE_ECB)
    padded_keyword = pad(keyword.encode())
    encrypted_keyword = cipher.encrypt(padded_keyword)
    return base64.b64encode(encrypted_keyword).decode('utf-8')

def encrypt_symmetric_key(user_key, sym_key):
    cipher = AES.new(user_key, AES.MODE_ECB)
    return base64.b64encode(cipher.encrypt(pad(sym_key))).decode('utf-8')

def decrypt_symmetric_key(user_key, enc_sym_key):
    cipher = AES.new(user_key, AES.MODE_ECB)
    sym_key_padded = cipher.decrypt(base64.b64decode(enc_sym_key))
    return unpad(sym_key_padded)

# Normalize text for consistent encryption
def normalize_text(text):
    return text.lower()

# Setup
master_key = get_random_bytes(32)

# User 1 and User 2 Keys
user1_id = 'user1'
user2_id = 'user2'
user1_key = derive_key(master_key, user1_id)
user2_key = derive_key(master_key, user2_id)
keyword_key = derive_key(master_key, 'keyword')

# Current date/time for binary data
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# User 1 encrypts data
documents = [
    {"title": "Document1", "description": "Description1", "binary": f"Document1 - Description1 // {current_datetime}"},
    {"title": "Document 2", "description": "Description2", "binary": f"Document2 - Description2 // {current_datetime}"},
    {"title": "Document 3", "description": "Description3", "binary": f"Document3 - Description3 // {current_datetime}"}
]

encrypted_documents = []
encrypted_indices = []
for doc in documents:
    sym_key_title = get_random_bytes(32)
    sym_key_desc = get_random_bytes(32)
    sym_key_binary = get_random_bytes(32)
    
    enc_title = encrypt_data(sym_key_title, doc["title"])
    enc_desc = encrypt_data(sym_key_desc, doc["description"])
    enc_binary = encrypt_data(sym_key_binary, doc["binary"])
    
    enc_sym_key_title_user1 = encrypt_symmetric_key(user1_key, sym_key_title)
    enc_sym_key_desc_user1 = encrypt_symmetric_key(user1_key, sym_key_desc)
    enc_sym_key_binary_user1 = encrypt_symmetric_key(user1_key, sym_key_binary)
    
    enc_sym_key_title_user2 = encrypt_symmetric_key(user2_key, sym_key_title)
    enc_sym_key_desc_user2 = encrypt_symmetric_key(user2_key, sym_key_desc)
    enc_sym_key_binary_user2 = encrypt_symmetric_key(user2_key, sym_key_binary)
    
    encrypted_documents.append({
        "enc_title": enc_title,
        "enc_desc": enc_desc,
        "enc_binary": enc_binary,
        "enc_sym_key_title_user1": enc_sym_key_title_user1,
        "enc_sym_key_desc_user1": enc_sym_key_desc_user1,
        "enc_sym_key_binary_user1": enc_sym_key_binary_user1,
        "enc_sym_key_title_user2": enc_sym_key_title_user2,
        "enc_sym_key_desc_user2": enc_sym_key_desc_user2,
        "enc_sym_key_binary_user2": enc_sym_key_binary_user2
    })
    
    encrypted_index = set()
    for field in [doc["title"], doc["description"], doc["binary"]]:
        normalized_field = normalize_text(field)
        for word in normalized_field.split():
            encrypted_index.add(encrypt_keyword(keyword_key, word))
    encrypted_indices.append(encrypted_index)

# Function to search encrypted data
def search_encrypted_data(keyword_key, user_key, keyword, encrypted_documents, encrypted_indices):
    matches = []
    normalized_keyword = normalize_text(keyword)
    keyword_enc = encrypt_keyword(keyword_key, normalized_keyword)
    print(f'Searching for: {keyword}')
    print(f'Encrypted keyword: {keyword_enc}')
    for i, enc_index in enumerate(encrypted_indices):
        print(f'Checking index {i}: {enc_index}')
        if keyword_enc in enc_index:
            matches.append(encrypted_documents[i])
            print(f'Match found in document index: {i}')
    return matches

# User 2 searches for a keyword in encrypted data
search_keyword = "Document"
matching_documents = search_encrypted_data(keyword_key, user2_key, search_keyword, encrypted_documents, encrypted_indices)
decrypted_documents = []
for doc in matching_documents:
    sym_key_title = decrypt_symmetric_key(user2_key, doc["enc_sym_key_title_user2"])
    sym_key_desc = decrypt_symmetric_key(user2_key, doc["enc_sym_key_desc_user2"])
    sym_key_binary = decrypt_symmetric_key(user2_key, doc["enc_sym_key_binary_user2"])
    
    title = decrypt_data(sym_key_title, doc["enc_title"])
    description = decrypt_data(sym_key_desc, doc["enc_desc"])
    binary = decrypt_data(sym_key_binary, doc["enc_binary"])
    
    decrypted_documents.append({"title": title, "description": description, "binary": binary})

# Output
print("Encrypted Documents:")
for doc in encrypted_documents:
    print(doc)
print("\nMatching Documents:")
for doc in matching_documents:
    print(doc)
print("\nDecrypted Documents:")
for doc in decrypted_documents:
    print(doc)


Searching for: Document
Encrypted keyword: XzmRc09NeBgtUmM0kbQnrA==
Checking index 0: {'S5B04ugEq+pVpEyaR4wZ2g==', 'G5908Kx7XMALTMXTY9uQ3Q==', 'Hr7l9vyDrQ2QSSfF7G40lA==', '9363sQ3iBUhbiNJAeJ2PWw==', '0BDuBqpSAa4ThZKnuF09ag==', 'WhsID77W9XJs4ytZjvig3w=='}
Checking index 1: {'S5B04ugEq+pVpEyaR4wZ2g==', 'XzmRc09NeBgtUmM0kbQnrA==', 'Hr7l9vyDrQ2QSSfF7G40lA==', '9363sQ3iBUhbiNJAeJ2PWw==', 'ZtwoJq+6SIZcY7VqHVuXnw==', '0BDuBqpSAa4ThZKnuF09ag==', '+a9LXmeWQcyBD2weWs6mSQ==', '/UgsN+Udw27h6UFRPYzNpA=='}
Match found in document index: 1
Checking index 2: {'NOoNVL9ztjStmBd1J+Nj9Q==', 'S5B04ugEq+pVpEyaR4wZ2g==', 'KjDTJHPd1UvWIHU5lebd8A==', 'XzmRc09NeBgtUmM0kbQnrA==', 'Hr7l9vyDrQ2QSSfF7G40lA==', '9363sQ3iBUhbiNJAeJ2PWw==', 'G93+wufETmuvdLHk9NZONA==', '0BDuBqpSAa4ThZKnuF09ag=='}
Match found in document index: 2
Encrypted Documents:
{'enc_title': 'ppg44Y0Eg6/D4lHnT+FNHqJYWEa1CrqZ4V7A0nJhkiI=', 'enc_desc': 'IsDSO4GeBe1bq07nWLidn8DzF9eUpGKHqOve/HSbuOo=', 'enc_binary': 'K+U5q2nsg4PiXFKEWiokyWEAPbLQmLBB8An

In [143]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Protocol.KDF import PBKDF2
import hashlib
import base64
from datetime import datetime

# Helper functions
def pad(data):
    pad_len = 16 - len(data) % 16
    return data + bytes([pad_len] * pad_len)

def unpad(data):
    pad_len = data[-1]
    return data[:-pad_len]

def derive_key(master_key, user_id):
    salt = hashlib.sha256(user_id.encode()).digest()
    return PBKDF2(master_key, salt, dkLen=32, count=1000)

def encrypt_data(sym_key, data):
    cipher = AES.new(sym_key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(data.encode()))
    return base64.b64encode(cipher.iv + ct_bytes).decode('utf-8')

def decrypt_data(sym_key, enc_data):
    enc_data = base64.b64decode(enc_data)
    iv = enc_data[:16]
    ct = enc_data[16:]
    cipher = AES.new(sym_key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(ct)).decode('utf-8')

def encrypt_keyword(key, keyword):
    cipher = AES.new(key, AES.MODE_ECB)
    padded_keyword = pad(keyword.encode())
    encrypted_keyword = cipher.encrypt(padded_keyword)
    return base64.b64encode(encrypted_keyword).decode('utf-8')

def encrypt_symmetric_key(user_key, sym_key):
    cipher = AES.new(user_key, AES.MODE_ECB)
    return base64.b64encode(cipher.encrypt(pad(sym_key))).decode('utf-8')

def decrypt_symmetric_key(user_key, enc_sym_key):
    cipher = AES.new(user_key, AES.MODE_ECB)
    sym_key_padded = cipher.decrypt(base64.b64decode(enc_sym_key))
    return unpad(sym_key_padded)

# Normalize text for consistent encryption
def normalize_text(text):
    return text.lower()

# Setup
master_key = get_random_bytes(32)

# User 1 and User 2 Keys
user1_id = 'user1'
user2_id = 'user2'
user1_key = derive_key(master_key, user1_id)
user2_key = derive_key(master_key, user2_id)

# Current date/time for binary data
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# User 1 encrypts data
documents = [
    {"title": "Document1", "description": "Description1", "binary": f"Document1 - Description1 // {current_datetime}"},
    {"title": "Document2", "description": "Description2", "binary": f"Document2 - Description2 // {current_datetime}"},
    {"title": "Document3", "description": "Description3", "binary": f"Document3 - Description3 // {current_datetime}"}
]

encrypted_documents = []
encrypted_indices_user1 = []
encrypted_indices_user2 = []

for doc in documents:
    sym_key_title = get_random_bytes(32)
    sym_key_desc = get_random_bytes(32)
    sym_key_binary = get_random_bytes(32)
    
    enc_title = encrypt_data(sym_key_title, doc["title"])
    enc_desc = encrypt_data(sym_key_desc, doc["description"])
    enc_binary = encrypt_data(sym_key_binary, doc["binary"])
    
    enc_sym_key_title_user1 = encrypt_symmetric_key(user1_key, sym_key_title)
    enc_sym_key_desc_user1 = encrypt_symmetric_key(user1_key, sym_key_desc)
    enc_sym_key_binary_user1 = encrypt_symmetric_key(user1_key, sym_key_binary)
    
    enc_sym_key_title_user2 = encrypt_symmetric_key(user2_key, sym_key_title)
    enc_sym_key_desc_user2 = encrypt_symmetric_key(user2_key, sym_key_desc)
    enc_sym_key_binary_user2 = encrypt_symmetric_key(user2_key, sym_key_binary)
    
    encrypted_documents.append({
        "enc_title": enc_title,
        "enc_desc": enc_desc,
        "enc_binary": enc_binary,
        "enc_sym_key_title_user1": enc_sym_key_title_user1,
        "enc_sym_key_desc_user1": enc_sym_key_desc_user1,
        "enc_sym_key_binary_user1": enc_sym_key_binary_user1,
        "enc_sym_key_title_user2": enc_sym_key_title_user2,
        "enc_sym_key_desc_user2": enc_sym_key_desc_user2,
        "enc_sym_key_binary_user2": enc_sym_key_binary_user2
    })
    
    encrypted_index_user1 = set()
    encrypted_index_user2 = set()
    
    for field in [doc["title"], doc["description"], doc["binary"]]:
        normalized_field = normalize_text(field)
        for word in normalized_field.split():
            encrypted_index_user1.add(encrypt_keyword(user1_key, word))
            encrypted_index_user2.add(encrypt_keyword(user2_key, word))
    
    encrypted_indices_user1.append(encrypted_index_user1)
    encrypted_indices_user2.append(encrypted_index_user2)

# Function to search encrypted data
def search_encrypted_data(user_key, keyword, encrypted_documents, encrypted_indices):
    matches = []
    normalized_keyword = normalize_text(keyword)
    keyword_enc = encrypt_keyword(user_key, normalized_keyword)
    print(f'Searching for: {keyword}')
    print(f'Encrypted keyword: {keyword_enc}')
    for i, enc_index in enumerate(encrypted_indices):
        print(f'Checking index {i}: {enc_index}')
        if keyword_enc in enc_index:
            matches.append(encrypted_documents[i])
            print(f'Match found in document index: {i}')
    return matches

# User 2 searches for a keyword in encrypted data
search_keyword = "Document2"
matching_documents = search_encrypted_data(user2_key, search_keyword, encrypted_documents, encrypted_indices_user2)
decrypted_documents = []
for doc in matching_documents:
    sym_key_title = decrypt_symmetric_key(user2_key, doc["enc_sym_key_title_user2"])
    sym_key_desc = decrypt_symmetric_key(user2_key, doc["enc_sym_key_desc_user2"])
    sym_key_binary = decrypt_symmetric_key(user2_key, doc["enc_sym_key_binary_user2"])
    
    title = decrypt_data(sym_key_title, doc["enc_title"])
    description = decrypt_data(sym_key_desc, doc["enc_desc"])
    binary = decrypt_data(sym_key_binary, doc["enc_binary"])
    
    decrypted_documents.append({"title": title, "description": description, "binary": binary})

# Output
print("Encrypted Documents:")
for doc in encrypted_documents:
    print(doc)
print("\nMatching Documents:")
for doc in matching_documents:
    print(doc)
print("\nDecrypted Documents:")
for doc in decrypted_documents:
    print(doc)


Searching for: Document2
Encrypted keyword: 7oGwiGGHKVcgOkDvfEjiqA==
Checking index 0: {'q0feZ8HDbfM1yAfqSkZVaQ==', 'twzGQpoIqccs2XIp13jXuQ==', 'EsyzUSSjqTXgelgU452tGA==', 'E3EscU04MHlLEZApVuAd2w==', 'ewnXPH7hCELJPzw0/kPvVA==', 'BFidrGJ7m9DqtE5U/WCZUw=='}
Checking index 1: {'q0feZ8HDbfM1yAfqSkZVaQ==', 'twzGQpoIqccs2XIp13jXuQ==', '7oGwiGGHKVcgOkDvfEjiqA==', 'ewnXPH7hCELJPzw0/kPvVA==', 'BFidrGJ7m9DqtE5U/WCZUw==', 'y3krGB+VJynTFSdiqzJfhg=='}
Match found in document index: 1
Checking index 2: {'q0feZ8HDbfM1yAfqSkZVaQ==', 'twzGQpoIqccs2XIp13jXuQ==', '3KRUhjS0swpDm5BjUnpL/Q==', 'X3Kcs/ZQIYYQ1/PD2kNX7A==', 'ewnXPH7hCELJPzw0/kPvVA==', 'BFidrGJ7m9DqtE5U/WCZUw=='}
Encrypted Documents:
{'enc_title': '7LH+M7UecFT75alblTYPhak8vyvl/A0O2aJUE1vcpSw=', 'enc_desc': '2f2OTGTgHi7znqzvZL5WR6cWHc5gMNIM0OrZMcqzqLE=', 'enc_binary': 'KIuYgBkbkuknjmMhYmtlFUrDFdWTtVEJzdpvDENs6jLcfPHhALtSsIRJvMWwzEBQ8JiM8y539uVd7gQ6NWqrng==', 'enc_sym_key_title_user1': 'EtKT1Owc29XmXtzQugqNjfdVR6eTmpsmre/c4EfqJRdgerxofkqrp9t16q1U

In [150]:
# maybe the same thing as above?

from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import hashlib
import base64
import os

# Helper functions
def derive_key(password, salt):
    return hashlib.pbkdf2_hmac('sha256', password.encode(), salt, 100000, dklen=32)

def encrypt_data(key, data):
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(data.encode(), AES.block_size))
    iv = cipher.iv
    return base64.b64encode(iv + ct_bytes).decode('utf-8')

def decrypt_data(key, enc_data):
    enc_data = base64.b64decode(enc_data)
    iv = enc_data[:AES.block_size]
    ct = enc_data[AES.block_size:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(ct), AES.block_size).decode('utf-8')

def encrypt_keyword(key, keyword):
    cipher = AES.new(key, AES.MODE_ECB)
    padded_keyword = pad(keyword.encode(), AES.block_size)
    encrypted_keyword = cipher.encrypt(padded_keyword)
    return base64.b64encode(encrypted_keyword).decode('utf-8')

# Normalize text for consistent encryption
def normalize_text(text):
    return text.lower()

# Setup master key and derive user keys
master_password = "supersecret"
salt = os.urandom(16)
master_key = derive_key(master_password, salt)
user1_key = derive_key("user1_password", salt)
user2_key = derive_key("user2_password", salt)

# Encrypt text and create searchable indexes
def encrypt_and_index(text, user_key):
    encrypted_text = encrypt_data(user_key, text)
    encrypted_index = set()
    normalized_text = normalize_text(text)
    for word in normalized_text.split():
        encrypted_index.add(encrypt_keyword(user_key, word))
    return encrypted_text, encrypted_index

documents = [
    "This is a sample text about Python programming language.",
    "Another text including Python and encryption techniques.",
    "A final example text that talks about data security."
]

encrypted_documents = []
indexes_user1 = []
indexes_user2 = []

for doc in documents:
    enc_doc, index_user1 = encrypt_and_index(doc, user1_key)
    _, index_user2 = encrypt_and_index(doc, user2_key)  # Same text but different index for user2
    encrypted_documents.append(enc_doc)
    indexes_user1.append(index_user1)
    indexes_user2.append(index_user2)

# Generate search token
def generate_search_token(keyword, key):
    return encrypt_keyword(key, keyword)

# Search encrypted indexes
def search_encrypted_indexes(search_token, indexes):
    matching_indexes = []
    for i, index in enumerate(indexes):
        if search_token in index:
            matching_indexes.append(i)
    return matching_indexes

# User 2 searches for a keyword in encrypted indexes
search_keyword = "python"
search_token = generate_search_token(search_keyword, user2_key)
matching_indexes = search_encrypted_indexes(search_token, indexes_user2)

# Decrypt matching documents
decrypted_documents = [decrypt_data(user1_key, encrypted_documents[i]) for i in matching_indexes]

# Output
print("Encrypted Documents:")
for doc in encrypted_documents:
    print(doc)
print("\nMatching Documents:")
for idx in matching_indexes:
    print(f"Document Index: {idx}")
print("\nDecrypted Documents:")
for doc in decrypted_documents:
    print(doc)


Encrypted Documents:
rkRXKqav56eQLOGWLthNrZc2vlM54NZKZ2GeGI+kiMrwcU9bdIU2NW+7ijWuKj3S0achxiSroMpzlv1ECAwfoIq70FPSp0Cpl5hOBMl7+J8=
E/wCgZ6YeIxnnN4YoeeuiineNAyvaWra/hrz2+J2w6it8OvjuVyMoH5cLbi3Pf6tWEwGiO/IwtXiUZXsIKeOiE7v+zTNekZavlR48tYp4lE=
/NbzyzHjNps+qTNsZADLYOnjsG67LXXppnVfWoPIjg9no5YZltIkFn8kOqfzLaK51UjP9gHB0G6adqtC+w8ona6mM3VZ3XtoCsdTxYlky3w=

Matching Documents:
Document Index: 0
Document Index: 1

Decrypted Documents:
This is a sample text about Python programming language.
Another text including Python and encryption techniques.


In [153]:
# gives document and index size

from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import hashlib
import base64
import os

# Helper functions
def derive_key(password, salt):
    return hashlib.pbkdf2_hmac('sha256', password.encode(), salt, 100000, dklen=32)

def encrypt_data(key, data):
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(data.encode(), AES.block_size))
    iv = cipher.iv
    return base64.b64encode(iv + ct_bytes).decode('utf-8')

def decrypt_data(key, enc_data):
    enc_data = base64.b64decode(enc_data)
    iv = enc_data[:AES.block_size]
    ct = enc_data[AES.block_size:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(ct), AES.block_size).decode('utf-8')

def encrypt_keyword(key, keyword):
    cipher = AES.new(key, AES.MODE_ECB)
    padded_keyword = pad(keyword.encode(), AES.block_size)
    encrypted_keyword = cipher.encrypt(padded_keyword)
    return base64.b64encode(encrypted_keyword).decode('utf-8')

# Normalize text for consistent encryption
def normalize_text(text):
    return text.lower()

# Setup master key and derive user keys
master_password = "supersecret"
salt = os.urandom(16)
master_key = derive_key(master_password, salt)
user1_key = derive_key("user1_password", salt)
user2_key = derive_key("user2_password", salt)

# Encrypt text and create searchable indexes
def encrypt_and_index(text, user_key):
    encrypted_text = encrypt_data(user_key, text)
    encrypted_index = set()
    normalized_text = normalize_text(text)
    for word in normalized_text.split():
        encrypted_index.add(encrypt_keyword(user_key, word))
    return encrypted_text, encrypted_index

documents = [
    "This is a sample text about Python programming language.",
    "Another text including Python and encryption techniques.",
    "A final example text that talks about data security."
]

encrypted_documents = []
indexes_user1 = []
indexes_user2 = []

for doc in documents:
    enc_doc, index_user1 = encrypt_and_index(doc, user1_key)
    _, index_user2 = encrypt_and_index(doc, user2_key)  # Same text but different index for user2
    encrypted_documents.append(enc_doc)
    indexes_user1.append(index_user1)
    indexes_user2.append(index_user2)
    
    # Debug: Print the size of the plaintext record
    print(f"Plaintext size for record: {len(doc.encode())} bytes")
    
    # Debug: Calculate the size of the index for this record
    index_size = sum(len(keyword.encode()) for keyword in index_user1)
    print(f"Size of all indices for the record: {index_size} bytes")

# Generate search token
def generate_search_token(keyword, key):
    return encrypt_keyword(key, keyword)

# Search encrypted indexes
def search_encrypted_indexes(search_token, indexes):
    matching_indexes = []
    for i, index in enumerate(indexes):
        if search_token in index:
            matching_indexes.append(i)
    return matching_indexes

# User 2 searches for a keyword in encrypted indexes
search_keyword = "Python"
search_token = generate_search_token(search_keyword, user2_key)
matching_indexes = search_encrypted_indexes(search_token, indexes_user2)

# Decrypt matching documents
decrypted_documents = [decrypt_data(user1_key, encrypted_documents[i]) for i in matching_indexes]

# Output
print("Encrypted Documents:")
for doc in encrypted_documents:
    print(doc)
print("\nMatching Documents:")
for idx in matching_indexes:
    print(f"Document Index: {idx}")
print("\nDecrypted Documents:")
for doc in decrypted_documents:
    print(doc)


Plaintext size for record: 7384 bytes
Size of all indices for the record: 10032 bytes
Plaintext size for record: 56 bytes
Size of all indices for the record: 168 bytes
Plaintext size for record: 52 bytes
Size of all indices for the record: 216 bytes
Encrypted Documents:
C2k0gxrBSDSljK7X8On5eFwdLw29OZkCNuCBZVqOUUQgQ8W1WaARjX/a3dVu4tJbIiAxRPzN1rnNF6SniKmQliGrH/NlsvxTx2CZ8VDfEmlCnhNe6S/pv2zlfoZ+pNCOfsk1/bSVKZ0d1N69umrpJY2+e1XRn427K8/bj6WxFvz1Ha9vBZUj3tLVN4tzVk6pDOXe3LFoHdBDRXha0K0IJRBPXHmWOx6KJuyJGX+dWWVLmfaJ14n48WnsfzTk/gPgr+ku5qdyCLnJqXYLvVwwSA7SWmYdZHLxjqlcJ4uh3BN83PGwfCptq1aTnh7+It4fXvyHCkE23oWEbOLW0Nwn2zoTy19A9hvQXArqEJSQqGfcRnTLRmBt4k7YKde+jmGIhz33p55DpEf4IpUGeii4wH/FIViiIw6kAPue7xfmILgHORwkSa5SzPiEKpHtqoqisQgWaxSRig+BSxkDU5AlsWJ/5I1Xl3E2EQdQMt+JQksn1Mb5dAdJgm43Q44bz+ME+ZLaO7qICzC7JNXK75uMD3oETeT8JVXJqvZLorZJvG9gNlpcaukRBok/1uF1plqZsnx4Wb6FJXesPX1XYN+t76wyJgX9hu968Fh3glzXYWhdtoWP+96PyYADRSEBLXjHaNZYioddY24wQ8nQ20AQgem/p4sEFP5h8Memn1kMywRj1LWroR2P8UjwPPM5CEOAj9fbXJeYA7znCCjuyJSkLUJ6T